In [ ]:
import os
import random
import subprocess

## Generate with FuzzGPT's few-shot (baseline)

Randomly select samples from the groups.

In [ ]:
base_path = "llm_generate"
prompt_path = os.path.join(base_path, "llm_prompt")
output_path = os.path.join(base_path, "llm_output")

all_samples = []
for category in ["gpu", "operator", "syntax", "typing"]:
    for sample in os.listdir(os.path.join(base_path, category)):
        if not sample.endswith(".txt"):
            continue
        all_samples.append(os.path.join(category, sample))
print(len(all_samples))

test_index = 0

while len(all_samples) > 0:
    sample1 = all_samples.pop(random.randrange(len(all_samples)))
    sample2 = all_samples.pop(random.randrange(len(all_samples)))
    sample3 = all_samples.pop(random.randrange(len(all_samples)))
    sample4 = all_samples.pop(random.randrange(len(all_samples)))
    samples = [sample1, sample2, sample3, sample4]
    print(samples)

    prompt_file = os.path.join(prompt_path, f"test_{test_index}.txt")
    with open(prompt_file, "w") as f:
        for sample in samples:
            sample_file = os.path.join(base_path, sample)
            with open(sample_file, "r") as f1:
                snippet = f1.read()
            f.write(f"{snippet}\n\n")

        f.write("# API:")

    out_file = os.path.join(output_path, f"test_{test_index}.txt")
    result = subprocess.run(
        [
            "llama-cli",
            "-m",
            "/path/to/model",
            "-c",
            "16000",
            "-n",
            "4096",
            "-ngl",
            "1",
            "-f",
            prompt_file,
        ],
        check=True,
        capture_output=True,
    )

    with open(os.path.join(output_path, out_file), "w") as f:
        f.write("# API:")
        f.write(result.stdout.decode("utf-8"))

    test_index += 1

## RQ1 evaluation

Run each generate test cases and collect the outputs in the `evaluation` directory.

Count and compare the valid ratio of the output test cases.

In [21]:
with open("evaluation/yanhui_results.txt", "r") as f:
    l1 = f.readline()
    total = int(l1.split(":")[-1])

    test_id = None
    fail_output = 0

    for line in f.readlines():
        if line.strip() == "":
            continue

        if test_id is None and line.startswith("#"):
            test_id = int(line.strip().split("#")[1][:-1])
            continue
        if test_id is not None:
            if line != "pass":
                fail_output += 1
            test_id = None

    pass_rate = (total - fail_output) / total
    print(f"YanHui Pass rate: {pass_rate}")

with open("evaluation/random_results.txt", "r") as f:
    l1 = f.readline()
    total = int(l1.split(":")[-1])

    test_id = None
    fail_output = 0

    for line in f.readlines():
        if line.strip() == "":
            continue

        if test_id is None and line.startswith("#"):
            test_id = int(line.strip().split("#")[1][:-1])
            continue
        if test_id is not None:
            if line != "pass":
                fail_output += 1
            test_id = None

    pass_rate = (total - fail_output) / total
    print(f"FuzzGPT-FS Pass rate: {pass_rate}")

YanHui Pass rate: 0.4583333333333333
FuzzGPT-FS Pass rate: 0.16666666666666666
